# Word2Vec

LIBROS .PDF UTILIZADOS, DESCARGARLOS Y COPIAR LA CARPETA EN EL MISMO DIRECTORIO DEL ARCHIVO DE LA NOTEBOOK.
https://drive.google.com/drive/folders/1n7c507Ty6CmqQjID--lJFA50zPsVln52?usp=sharing

Instalar:
pip install pypdf2 -q
pip install tqdm -q

## 1. Importar Librerias

In [ ]:
import string
from gensim.models import Word2Vec
import PyPDF2                        #LECTOR DE .PDF

##### LIBROS (sociologia, historia)

IMPORTAR LIBRERIAS

In [ ]:
import os                       #MANEJO DE ARCHIVOS
from tqdm import tqdm           #MUESTRA BARRA DE PROGRESO

SE RENOMBRAN LOS DOCUMENTOS .PDF EN NOMBRES CONSECUTIVOS.
SOLO SE PODRÁN RENOMBRAR UNA VEZ

In [ ]:
# Ruta de la carpeta que contiene los archivos PDF
path = 'libros'

# Lista de archivos en la carpeta
files = [f for f in os.listdir(path) if f.endswith('.pdf')]

# Renombrar los archivos
for i, filename in enumerate(sorted(files)):
    old_path = os.path.join(path, filename)
    new_path = os.path.join(path, f"archivo {i}.pdf")
    os.rename(old_path, new_path)

print("Archivos renombrados con éxito.")

FUNCIÓN QUE OBTIENE LOS TEXTOS DE LOS .PDF -> texto

In [ ]:
def extraer_texto_desde_pdf(ruta_archivo):
    with open(ruta_archivo, 'rb') as archivo:
        lector = PyPDF2.PdfReader(archivo)
        texto = ""
        for pagina in range(len(lector.pages)):
            texto += lector.pages[pagina].extract_text()
    return texto

ARREGLO QUE REÚNE EN UNA SOLA VARIABLE "todos_los_textos", TODA LA INFORMACIÓN EXTRAIDA DE LOS .PDFs
ANTE UN "NO RECONOCIMIENTO DE TEXTO ILEGIBLE" -> try - except

In [ ]:
ruta_carpeta = "libros"
todos_los_textos = []

for archivo in tqdm(os.listdir(ruta_carpeta)):
    if archivo.endswith('.pdf'):
        ruta_completa = os.path.join(ruta_carpeta, archivo)
        try:
            documento = extraer_texto_desde_pdf(ruta_completa)
            todos_los_textos.append(documento)
        except Exception as e:
            print(f"Error al procesar {archivo}: {e}")

CANTIDAD DE ARCHIVOS

In [ ]:
len(todos_los_textos)

SE SEPARA EL TEXTO POR EL CARACTER ESPECIFICADO = '.'

In [ ]:
oraciones_totales = []
tamaño = 0

for documento in todos_los_textos:
    tamaño = tamaño + len(documento)
    oraciones = documento.split('.')
    oraciones_totales.extend(oraciones) 

In [ ]:
print(f"Número de carácteres: {tamaño}")

In [ ]:
print("Cantidad de oraciones:", len(oraciones_totales))

QUITAMOS PUNTUACIONES Y TRANSFORMAMOS A MINÚSCULAS

In [ ]:
oraciones_limpias = []
for oracion in oraciones_totales:
    # Eliminar puntuación y dividir por espacios
    tokens = oracion.translate(str.maketrans('', 
                                    '', string.punctuation)).split()
    # Convertir a minúsculas
    tokens = [word.lower() for word in tokens if word.isalpha()]
    if tokens:  # Añadir solo si hay tokens
        oraciones_limpias.append(tokens)

MOSTRAMOS VECTOR N°:

In [ ]:
oraciones_limpias[25000]

ENTRENAMIENTO DEL MODELO.
TAMAÑO VECTOR=500, CANT. PALABRAS ANTES/DESPUÉS=5, MÍNIMO DE REPETICIONES POR PALABRA=3, PROCESADORES O NÚCLEOS (CPU)=8

In [ ]:
# Entrenar el modelo Word2Vec
model = Word2Vec(sentences=oraciones_limpias, 
                 vector_size=500, window=5, min_count=3, workers=8)

PALABRAS CERCANAS. INTRODUCIR DISTINTAS PALABRAS Y VERIFICAR LA CERCANIA DEL RESULTADO.
topn=5 (cantida de resultados)
UNO O MAS RESULTADOS DEBERÍA TENER RELACIÓN. DEPENDE DE LOS DATOS DE ENTRENAMIENTO.

In [ ]:
palabras_cercanas = model.wv.most_similar("hombre", topn=5)
palabras_cercanas

In [ ]:
palabras_cercanas = model.wv.most_similar("mujer", topn=5)
palabras_cercanas

In [ ]:
palabras_cercanas = model.wv.most_similar("familia", topn=5)
palabras_cercanas

In [ ]:
palabras_cercanas = model.wv.most_similar("país", topn=5)
palabras_cercanas

In [ ]:
palabras_cercanas = model.wv.most_similar("iglesia", topn=5)
palabras_cercanas

In [ ]:
palabras_cercanas = model.wv.most_similar("guerra", topn=5)
palabras_cercanas

In [ ]:
palabras_cercanas = model.wv.most_similar("ley", topn=5)
palabras_cercanas

In [ ]:
palabras_cercanas = model.wv.most_similar("trabajo", topn=5)
palabras_cercanas

SE GUARDA EL MODELO. embedding y binary

In [ ]:
model.wv.save_word2vec_format('emb_libros.txt', binary=False)
model.wv.save_word2vec_format('emb_libros.bin', binary=True)

SE CARGA EL MODELO

In [ ]:
from gensim.models import KeyedVectors

embeddings_cargados = KeyedVectors.load_word2vec_format('emb_libros.txt', binary=False)
# O si fue guardado en formato binario:
# embeddings_cargados = KeyedVectors.load_word2vec_format('embeddings.bin', binary=True)
embeddings_cargados

FUNCIÓN DE ANALOGÍA. SE CARGA EL EMBEDDING Y SE LE PASA 3 PALABRAS PARA VERIFICAR SU ANALOGÍA.
MÉTODO most_similar

In [ ]:
def analogias(v1, v2, v3):
    similitud = embeddings_cargados.most_similar(positive=[v1, v3], negative=[v2])
    print(f"{v1} es a {v2} como {similitud[0][0]} es a {v3}") 

In [ ]:
analogias('mercado', 'economía', 'trabajo')

In [ ]:
analogias('política', 'partido', 'país')

In [ ]:
analogias('tecnología', 'avance', 'futuro')

In [ ]:
analogias('fuerza', 'máquina', 'hombre')

EL RESULTADO DEPENDERÁ DE LOS DATOS DE ENTRENAMIENTO.
EN ESTE CASO SE UTILIZARON TEXTOS DE 'HISTORIA' Y 'SOCIOLOGÍA', PODRÍA CAMBIARSE EL ÁREA DE APLICACIÓN O SUMARLE OTRA.